In [ ]:
import pandas as pd

import pandas2neo4j

from models import Address, Person, Publication

### Connect to the database

In [3]:
HOST = "wiatrak2:7687"
USERNAME = "neo4j"
PASSWORD = "s3cr3t"

In [4]:
pd_graph = pandas2neo4j.pandas_graph.PandasGraph(f"bolt://{USERNAME}:{PASSWORD}@{HOST}")

### Create `uuid` uniqueness contraint

In [ ]:
pd_graph.schema.create_uniqueness_constraint("Address", "uuid")
pd_graph.schema.create_uniqueness_constraint("Person", "uuid")
pd_graph.schema.create_uniqueness_constraint("Publication", "uuid")

### Load tables into `pd.DataFrame`s

In [5]:
addresses_df = pd.read_csv("data/addresses.csv")
people_df = pd.read_csv("data/people.csv")
publications_df = pd.read_csv("data/publications.csv")

addresses_people_df = pd.read_csv("data/addresses_people.csv")
addresses_publications_df = pd.read_csv("data/addresses_publications.csv")
people_publications_df = pd.read_csv("data/people_publications.csv")

### Create database nodes from tables rows

In [6]:
addresses_models = pd_graph.create_nodes_from_dataframe(addresses_df, Address)

In [7]:
people_models = pd_graph.create_nodes_from_dataframe(people_df, Person)

In [8]:
publications_models = pd_graph.create_nodes_from_dataframe(publications_df, Publication, chunk_size=1000)

### Create relationships

In [11]:
addresses_people = pd_graph.create_relationships_from_dataframe(
    addresses_people_df,
    "ADDRESS", 
    Address, 
    Person, 
    "address_uuid",
    "person_uuid", 
    chunk_size=1000,
)

In [13]:
addresses_people[0]

ADDRESS(Node('Address', city='South Nicoleburgh', country='Austria', lat=59.413341, lon=57.306847, uuid=0), Node('Person', company='Schaefer-Morris', email='ashleypowell@chang.biz', firstname='Wendy', lastname='Ramos', phone_number='+1-016-246-2240x0680', uuid=0))

In [16]:
addresses_publications = pd_graph.create_relationships_from_dataframe(
    addresses_publications_df,
    "AFFILIATION", 
    Address, 
    Publication, 
    "address_uuid",
    "publication_uuid", 
    chunk_size=1000,
)

In [18]:
people_publications = pd_graph.create_relationships_from_dataframe(
    people_publications_df,
    "AUTHOR", 
    Person, 
    Publication, 
    "person_uuid", 
    "publication_uuid",
    chunk_size=1000,
)

![graph](img/graph_demo.png)

### Get models from graph nodes

In [25]:
person_models = pd_graph.get_graph_models(Person)
print(person_models[0].lastname)

Ramos


In [ ]:
models_for_df = pd_graph.get_models_for_dataframe(addresses_df, Address, "uuid")

In [40]:
models_for_df[:10]

,uuid,Address
0,0,<Address uuid=0>
1,1,<Address uuid=1>
2,2,<Address uuid=2>
3,3,<Address uuid=3>
4,4,<Address uuid=4>
5,5,<Address uuid=5>
6,6,<Address uuid=6>
7,7,<Address uuid=7>
8,8,<Address uuid=8>
9,9,<Address uuid=9>


### Contruct `DataFrame` for graph nodes

In [47]:
addresses_in_graph_df = pd_graph.get_dataframe_for_label("Address")
addresses_in_graph_df[:10]

,country,lon,city,uuid,lat
0,Bangladesh,-123.351771,New Jamie,2,-0.412780
1,Kiribati,-62.894170,Vincentborough,3,73.658648
2,Uganda,146.282126,Colinstad,4,34.262528
3,Netherlands Antilles,16.828492,Jonesside,5,40.344708
4,Antigua and Barbuda,81.919966,West Ryanburgh,6,177.766069
5,Saint Lucia,-138.363989,Fosterville,7,41.105460
6,Comoros,-98.196152,East Lindsay,8,42.518458
7,Luxembourg,90.866859,Martinezland,9,149.878195
8,Albania,-14.234085,New Larry,10,-41.463104
9,Argentina,-166.355181,Johnmouth,11,62.624384


In [48]:
people_in_graph_df = pd_graph.get_dataframe_for_models(Person, columns=["uuid", "firstname", "lastname"])
people_in_graph_df

,uuid,firstname,lastname
0,0,Wendy,Ramos
1,1,Christopher,Madden
2,2,Marcus,Guerrero
3,3,Brandy,Hernandez
4,4,Phillip,Norton
...,...,...,...
995,995,Grant,Little
996,996,Lori,Harrington
997,997,Terri,Holland
998,998,Mackenzie,Lowe


### Get relationships containing nodes

In [ ]:
people_for_df = pd_graph.get_models_for_dataframe(people_df, Person, "uuid")
people_addresses = pd_graph.get_relationships("ADDRESS", people_for_df["Person"])

In [63]:
people_addresses[0]

ADDRESS(Node('Address', city='East Maria', country='Monaco', lat=-153.405541, lon=143.756567, uuid=26), Node('Person', company='Hayes-King', email='joe54@yahoo.com', firstname='Brent', lastname='Clark', phone_number='6221424285', uuid=949))

### Match existing relationships with `DataFrame`

In [50]:
relationships_in_graph = pd_graph.get_relationships_for_dataframe(
    addresses_publications_df, "AFFILIATION", Address, Publication, "address_uuid", "publication_uuid"
)

In [53]:
relationships_in_graph

,address_uuid,publication_uuid,AFFILIATION
0,0.0,0.0,{}
1,0.0,86.0,{}
2,0.0,25.0,{}
3,0.0,77.0,{}
4,1.0,1.0,{}
...,...,...,...
195,48.0,79.0,{}
196,49.0,49.0,{}
197,49.0,17.0,{}
198,49.0,48.0,{}


### Get `DataFrame` with existing relationships

In [65]:
addresses_in_graph = pd_graph.get_dataframe_for_relationship(
    "ADDRESS", from_node_property="uuid", to_node_property="uuid"
)

In [67]:
addresses_in_graph

,uuid_from,uuid_to
0,2,498
1,2,896
2,2,358
3,2,798
4,2,730
...,...,...
490,1,711
491,1,173
492,1,126
493,1,941
